In [1]:
# 1. 데이터 다운로드 & 압축 해제
!wget -O data.tar.gz "https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz"
!tar -xzf data.tar.gz

# 2. 파일 확인
!ls -la data/

--2026-01-28 07:46:14--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 3.5.187.45, 52.219.206.87, 3.5.189.87, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|3.5.187.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3280795723 (3.1G) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   3.05G  43.3MB/s    in 73s     

2026-01-28 07:47:27 (42.7 MB/s) - ‘data.tar.gz’ saved [3280795723/3280795723]

total 16
drwxr-xr-x 4 root root 4096 Dec 17  2024 .
drwxr-xr-x 1 root root 4096 Jan 28 07:47 ..
drwxr-xr-x 4 root root 4096 Jan 27  2024 datasets
drwxr-xr-x 5 root root 4096 Dec 17  2024 pseudo_label


In [2]:
!pip install albumentations segmentation-models-pytorch tqdm opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.4 MB/s eta 0:00:00


In [2]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc

# --- 1. A100 최적화 설정 ---
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

# --- 2. 경로 및 하이퍼파라미터 ---
BASE_PATH = './data/datasets'
TRAIN_IMG_DIR = os.path.join(BASE_PATH, 'images/train')
VAL_IMG_DIR = os.path.join(BASE_PATH, 'images/val')
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')

TRAIN_JSON = os.path.join(BASE_PATH, 'jsons/train.json')
VAL_JSON = os.path.join(BASE_PATH, 'jsons/val.json')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# [A100 최적화 - 안정적인 설정]
BATCH_SIZE = 16
ACCUMULATION_STEPS = 2      # 실효 배치 = 32
RESIZE_TARGET = 512         # 768은 메모리 많이 먹음
EPOCHS = 20
LEARNING_RATE = 1e-4

# --- 3. 데이터셋 클래스 ---
class ReceiptDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None, is_test=False):
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            return self.__getitem__((idx + 1) % len(self))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        if not self.is_test:
            mask = np.zeros((h, w), dtype=np.float32)
            words = self.data[img_name].get('words', {})
            for word_id in words:
                points = np.array(words[word_id]['points'], dtype=np.int32)
                cv2.fillPoly(mask, [points], 1)
            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                return augmented['image'], augmented['mask']
        else:
            if self.transform:
                augmented = self.transform(image=image)
                return augmented['image'], img_name, (h, w)
        return image

# --- 4. 전처리 ---
train_transform = A.Compose([
    A.Resize(RESIZE_TARGET, RESIZE_TARGET),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(RESIZE_TARGET, RESIZE_TARGET),
    A.Normalize(),
    ToTensorV2()
])

# --- 5. 메인 함수 ---
def main():
    # GPU 정보 출력
    print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    torch.cuda.empty_cache()
    gc.collect()

    train_ds = ReceiptDataset(TRAIN_IMG_DIR, TRAIN_JSON, transform=train_transform)
    val_ds = ReceiptDataset(VAL_IMG_DIR, VAL_JSON, transform=val_transform)

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
        persistent_workers=True  # A100에서 효과적
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True
    )

    model = smp.UnetPlusPlus(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1
    ).to(DEVICE)

    # 모델을 channels_last 포맷으로 (A100 텐서코어 최적화)
    model = model.to(memory_format=torch.channels_last)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    criterion = smp.losses.DiceLoss(mode='binary')
    scaler = GradScaler('cuda')

    best_val_loss = float('inf')

    print(f"🚀 A100 학습 시작")
    print(f"   Batch: {BATCH_SIZE} × Accumulation: {ACCUMULATION_STEPS} = 실효 {BATCH_SIZE * ACCUMULATION_STEPS}")
    print(f"   Image Size: {RESIZE_TARGET}×{RESIZE_TARGET}")

    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0
        optimizer.zero_grad(set_to_none=True)

        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}")
        for step, (images, masks) in pbar:
            # channels_last 포맷 적용
            images = images.to(DEVICE, non_blocking=True, memory_format=torch.channels_last)
            masks = masks.to(DEVICE, non_blocking=True).unsqueeze(1)

            with autocast('cuda', dtype=torch.bfloat16):  # A100은 bfloat16이 더 안정적
                outputs = model(images)
                loss = criterion(outputs, masks) / ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (step + 1) % ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

            train_loss += loss.item() * ACCUMULATION_STEPS
            pbar.set_postfix({'loss': f'{loss.item() * ACCUMULATION_STEPS:.4f}'})

        scheduler.step()
        torch.cuda.empty_cache()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in tqdm(val_loader, desc="Val", leave=False):
                images = images.to(DEVICE, non_blocking=True, memory_format=torch.channels_last)
                masks = masks.to(DEVICE, non_blocking=True).unsqueeze(1)
                with autocast('cuda', dtype=torch.bfloat16):
                    outputs = model(images)
                    loss = criterion(outputs, masks)
                val_loss += loss.item()

        avg_val = val_loss / len(val_loader)
        avg_train = train_loss / len(train_loader)
        lr = scheduler.get_last_lr()[0]
        print(f"📊 Epoch {epoch}: Train {avg_train:.4f}, Val {avg_val:.4f}, LR {lr:.6f}")

        if avg_val < best_val_loss:
            best_val_loss = avg_val
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"   ✅ Best model saved!")

        # 메모리 사용량 출력
        mem_used = torch.cuda.max_memory_allocated() / 1e9
        print(f"   💾 Peak Memory: {mem_used:.1f} GB")
        torch.cuda.reset_peak_memory_stats()

    print("✅ 학습 완료!")

if __name__ == "__main__":
    main()

🖥️ GPU: NVIDIA A100-SXM4-40GB
💾 GPU Memory: 42.5 GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

🚀 A100 학습 시작
   Batch: 16 × Accumulation: 2 = 실효 32
   Image Size: 512×512


...

📊 Epoch 18: Train 0.0768, Val 0.0911, LR 0.000002
   ✅ Best model saved!
   💾 Peak Memory: 8.6 GB


Epoch 19: 100%|██████████| 204/204 [01:05<00:00,  3.12it/s, loss=0.0719]


📊 Epoch 19: Train 0.0761, Val 0.0911, LR 0.000001
   ✅ Best model saved!
   💾 Peak Memory: 8.6 GB


Epoch 20: 100%|██████████| 204/204 [01:05<00:00,  3.11it/s, loss=0.0889]


📊 Epoch 20: Train 0.0760, Val 0.0911, LR 0.000000
   ✅ Best model saved!
   💾 Peak Memory: 8.6 GB
✅ 학습 완료!


In [6]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm

# --- 설정 ---
BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')
MODEL_PATH = 'best_model.pth'
OUTPUT_CSV = 'submission.csv'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RESIZE_TARGET = 512
BATCH_SIZE = 16
THRESHOLD = 0.5

# --- 테스트 데이터셋 ---
class TestDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        if self.transform:
            augmented = self.transform(image=image)
            return augmented['image'], img_name, (h, w)
        return image, img_name, (h, w)

# --- 마스크 → 폴리곤 변환 ---
def mask_to_polygons(mask, min_area=100):
    polygons = []
    contours, _ = cv2.findContours(
        mask.astype(np.uint8),
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area:
            continue

        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) >= 4:
            points = approx.reshape(-1, 2).tolist()
            polygons.append(points)

    return polygons

# --- 폴리곤 → 문자열 변환 ---
def polygons_to_string(polygons):
    """[[x1,y1],[x2,y2],...] 형태를 문자열로 변환"""
    if not polygons:
        return ""

    result = []
    for poly in polygons:
        # [[x1,y1],[x2,y2],...] → "x1 y1 x2 y2 ..."  (공백 구분!)
        coords = []
        for point in poly:
            coords.append(f"{int(point[0])} {int(point[1])}")
        result.append(" ".join(coords))

    # 여러 폴리곤은 | 로 구분
    return "|".join(result)

# --- 추론 함수 ---
def inference():
    print(f"🔍 추론 시작")

    # sample_submission 읽기
    sample_df = pd.read_csv(SAMPLE_SUB)
    print(f"📄 Sample submission 형식:")
    print(f"   Columns: {list(sample_df.columns)}")
    print(f"   Shape: {sample_df.shape}")
    print(f"   Sample:\n{sample_df.head(3)}")

    # 전처리
    test_transform = A.Compose([
        A.Resize(RESIZE_TARGET, RESIZE_TARGET),
        A.Normalize(),
        ToTensorV2()
    ])

    # 데이터 로더
    test_ds = TestDataset(TEST_IMG_DIR, TEST_JSON, transform=test_transform)
    test_loader = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # 모델 로드
    model = smp.UnetPlusPlus(
        encoder_name="resnet34",
        encoder_weights=None,
        in_channels=3,
        classes=1
    ).to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model = model.to(memory_format=torch.channels_last)
    model.eval()

    # 결과 저장용 딕셔너리
    predictions = {}

    with torch.no_grad():
        for images, img_names, orig_sizes in tqdm(test_loader, desc="Inference"):
            images = images.to(DEVICE, memory_format=torch.channels_last)

            with autocast('cuda', dtype=torch.bfloat16):
                outputs = model(images)

            preds = torch.sigmoid(outputs).float().cpu().numpy()

            for i, img_name in enumerate(img_names):
                orig_h, orig_w = orig_sizes[0][i].item(), orig_sizes[1][i].item()

                # 원본 크기로 리사이즈
                pred_mask = preds[i, 0]
                pred_mask = cv2.resize(pred_mask, (orig_w, orig_h))

                # 이진화
                binary_mask = (pred_mask > THRESHOLD).astype(np.uint8)

                # 폴리곤 추출
                polygons = mask_to_polygons(binary_mask)

                # 이미지명을 키로 저장
                predictions[img_name] = polygons

    # --- sample_submission 형식에 맞춰 저장 ---
    result_df = sample_df.copy()

    # 컬럼명 확인 후 자동 매핑
    id_col = sample_df.columns[0]  # 첫 번째 컬럼 = ID/이미지명
    pred_col = sample_df.columns[1] if len(sample_df.columns) > 1 else 'prediction'  # 두 번째 컬럼 = 예측값

    print(f"\n📝 제출 파일 생성 중...")
    print(f"   ID 컬럼: {id_col}")
    print(f"   예측 컬럼: {pred_col}")

    for idx, row in result_df.iterrows():
        img_id = row[id_col]

        # 이미지명 매칭 (확장자 유무에 따라 처리)
        if img_id in predictions:
            polygons = predictions[img_id]
        elif f"{img_id}.jpg" in predictions:
            polygons = predictions[f"{img_id}.jpg"]
        elif f"{img_id}.png" in predictions:
            polygons = predictions[f"{img_id}.png"]
        else:
            polygons = []

        # 폴리곤을 문자열로 변환하여 저장
        result_df.at[idx, pred_col] = polygons_to_string(polygons)

    # CSV 저장
    result_df.to_csv(OUTPUT_CSV, index=False)

    print(f"\n✅ 제출 파일 저장 완료: {OUTPUT_CSV}")
    print(f"   총 이미지: {len(result_df)}개")
    print(f"\n📊 결과 미리보기:")
    print(result_df.head(5))

if __name__ == "__main__":
    inference()

🔍 추론 시작
📄 Sample submission 형식:
   Columns: ['filename', 'polygons']
   Shape: (413, 2)
   Sample:
                                   filename  \
0  drp.en_ko.in_house.selectstar_003883.jpg   
1  drp.en_ko.in_house.selectstar_000132.jpg   
2  drp.en_ko.in_house.selectstar_001796.jpg   

                                            polygons  
0  10 50 100 50 100 150 10 150|110 150 200 150 20...  
1  10 50 100 50 100 150 10 150|110 150 200 150 20...  
2  10 50 100 50 100 150 10 150|110 150 200 150 20...  


Inference: 100%|██████████| 26/26 [00:03<00:00,  8.08it/s]


📝 제출 파일 생성 중...
   ID 컬럼: filename
   예측 컬럼: polygons

✅ 제출 파일 저장 완료: submission.csv
   총 이미지: 413개

📊 결과 미리보기:
                                   filename  \
0  drp.en_ko.in_house.selectstar_003883.jpg   
1  drp.en_ko.in_house.selectstar_000132.jpg   
2  drp.en_ko.in_house.selectstar_001796.jpg   
3  drp.en_ko.in_house.selectstar_001202.jpg   
4  drp.en_ko.in_house.selectstar_002858.jpg   

                                            polygons  
0  392 1145 474 1149 493 1140 439 1135 398 1139|4...  
1  648 1131 648 1176 730 1178 730 1137|334 1023 3...  
2  236 1052 248 1174 469 1181 508 1131 633 1184 5...  
3  513 1223 529 1226 538 1212 657 1210 662 1202 5...  
4  477 1216 476 1228 555 1233 559 1226 556 1212|3...  


In [5]:
import pandas as pd
df = pd.read_csv('./data/datasets/sample_submission.csv')
print(df.head())
print(df.columns.tolist())
print(df.dtypes)

                                   filename  \
0  drp.en_ko.in_house.selectstar_003883.jpg   
1  drp.en_ko.in_house.selectstar_000132.jpg   
2  drp.en_ko.in_house.selectstar_001796.jpg   
3  drp.en_ko.in_house.selectstar_001202.jpg   
4  drp.en_ko.in_house.selectstar_002858.jpg   

                                            polygons  
0  10 50 100 50 100 150 10 150|110 150 200 150 20...  
1  10 50 100 50 100 150 10 150|110 150 200 150 20...  
2  10 50 100 50 100 150 10 150|110 150 200 150 20...  
3  10 50 100 50 100 150 10 150|110 150 200 150 20...  
4  10 50 100 50 100 150 10 150|110 150 200 150 20...  
['filename', 'polygons']
filename    object
polygons    object
dtype: object
